<a href="https://colab.research.google.com/github/day28645/Artificial-Intelligence/blob/main/Lab_week7_Naive_Bayes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

\begin{array}{|c|c|} \hline
 & Spam & & No Spam \\ \hline
Total & 25 & & 75 \\
Buy & 20 & 4/5 & 5 & 1/15 \\
Cheap & 15 & 3/5 & 10 & 2/15 \\
Work & 5 & 1/5 & 30 & 2/5 \\
Free & 20 & 4/5 & 7 & 7/75 \\
Buy, Cheap, Work, \text& Free  & 48/25 & 48/625 & 28/1125 & 28/84375 \\ \hline
\end{array}

\begin{align*}
P(spam \ if \ "Buy, Cheap, Work, \text& Free") &= \frac{48/25}{48/25+28/1125} \\
&= \frac{540}{547} \\
&= 98.72 \text% \\
\end{align*}

# Assignment 2

\begin{array}{|c|c|} \hline
 & Spam & & No Spam \\ \hline
Total & 25 & & 75 \\
Buy & 20 & 4/5 & 5 & 1/15 \\
Cheap & 15 & 3/5 & 10 & 2/15 \\
Work & 5 & 1/5 & 30 & 2/5 \\
Free & 20 & 4/5 & 7 & 7/75 \\
Will & 4 & 4/25 & 40 & 8/15 \\
Buy, Cheap, Work, Free, \text&Will & 192/625 & 192/15625 & 224/16875 & 224/1265625 \\ \hline
\end{array}

\begin{align*}
P(spam \ if \ "Buy, Cheap, Work, Free, \text&Will") &= \frac{192/625}{192/625+224/16875} \\
&= \frac{162}{169} \\
&= 95.86 \text% \\
\end{align*}

# Code

https://github.com/Midvel/medium_jupyter_notes/blob/master/naive_bayes_filter/bayes-classificator.ipynb

https://towardsdatascience.com/how-to-build-and-apply-naive-bayes-classification-for-spam-filtering-2b8d3308501

In [ ]:
import pandas as pd

Dataset from https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [ ]:
sms_data = pd.read_csv('/content/drive/MyDrive/COS3109/SMSSpamCollection', header=None, sep='\t', names=['Label', 'SMS'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sms_data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
sms_data.shape

(5572, 2)

In [ ]:
sms_data.loc[30,:]

Label                                                  ham
SMS      Wait that's still not all that clear, were you...
Name: 30, dtype: object

In [ ]:
sms_data.loc[30,'SMS']

"Wait that's still not all that clear, were you not sure about me being sarcastic or that that's why x doesn't want to live with us"

In [ ]:
sms_data.groupby('Label').count()

,SMS
Label,
ham,4825
spam,747


Prepare data

In [ ]:
sms_data_clean = sms_data.copy()

In [ ]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()

<ipython-input-14-b182ea16a502>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()


In [ ]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.lower()

In [ ]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.split()

In [ ]:
sms_data_clean['SMS'].head()

0    [go, until, jurong, point, crazy, available, o...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, don, t, think, he, goes, to, usf, he,...
Name: SMS, dtype: object

In [ ]:
sms_data.loc[2,'SMS']

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [ ]:
sms_data_clean.loc[2,'SMS']

['free',
 'entry',
 'in',
 '2',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 '21st',
 'may',
 '2005',
 'text',
 'fa',
 'to',
 '87121',
 'to',
 'receive',
 'entry',
 'question',
 'std',
 'txt',
 'rate',
 't',
 'c',
 's',
 'apply',
 '08452810075over18',
 's']

In [ ]:
sms_data_clean['Label'].value_counts() / sms_data.shape[0] * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Split to train and test data

In [ ]:
train_data = sms_data_clean.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = sms_data_clean.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

In [ ]:
train_data['Label'].value_counts() / train_data.shape[0] * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [ ]:
train_data.shape

(4458, 2)

In [ ]:
test_data['Label'].value_counts() / test_data.shape[0] * 100

ham     86.983842
spam    13.016158
Name: Label, dtype: float64

In [ ]:
test_data.shape

(1114, 2)

In [ ]:
test_data.head()

,Label,SMS
0,ham,"[aight, should, i, just, plan, to, come, up, l..."
1,ham,"[die, i, accidentally, deleted, e, msg, i, sup..."
2,spam,"[welcome, to, uk, mobile, date, this, msg, is,..."
3,ham,"[this, is, wishing, you, a, great, day, moji, ..."
4,ham,"[thanks, again, for, your, reply, today, when,..."


Prepare vocabulary - the list fo all the words from the dataset

In [ ]:
vocabulary = list(set(train_data['SMS'].sum()))

In [ ]:
vocabulary[11:20]

['wipro',
 'geoenvironmental',
 'jsut',
 'laden',
 'rang',
 'siguviri',
 'notebook',
 'follow',
 '08081263000']

In [ ]:
len(vocabulary)

7783

Calculate frequencies fo the words for each message

In [ ]:
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)

In [ ]:
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [ ]:
train_data.head()

,Label,SMS,trust,sex,residency,pants,talkbut,store,frndshp,ge,...,upgrading,enc,howu,3510i,25p,entire,0quit,datebox1282essexcm61xn,soil,iknow
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data['i']

0       0
1       0
2       0
3       0
4       1
       ..
4453    1
4454    5
4455    0
4456    4
4457    0
Name: i, Length: 4458, dtype: int64

Calculate values for the Bayes formula

In [ ]:
alpha = 1

In [ ]:
Nvoc = len(train_data.columns) - 3

In [ ]:
Pspam = train_data['Label'].value_counts()['spam'] / train_data.shape[0]

In [ ]:
Pham = train_data['Label'].value_counts()['ham'] / train_data.shape[0]

In [ ]:
Nspam = train_data.loc[train_data['Label'] == 'spam', 'SMS'].apply(len).sum()

In [ ]:
Nham = train_data.loc[train_data['Label'] == 'ham', 'SMS'].apply(len).sum()

In [ ]:
Nspam

15190

In [ ]:
Nham

57237

In [ ]:
def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1

In [ ]:
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

In [ ]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [ ]:
classify('secret')

'ham'

In [ ]:
classify(['secret', 'source', 'of', 'infinite', 'power'])

'ham'

In [ ]:
classify(['free', 'win', 'welcom'])

'spam'

Test data

In [ ]:
test_data['predicted'] = test_data['SMS'].apply(classify)

In [ ]:
test_data.head()

,Label,SMS,predicted
0,ham,"[aight, should, i, just, plan, to, come, up, l...",ham
1,ham,"[die, i, accidentally, deleted, e, msg, i, sup...",ham
2,spam,"[welcome, to, uk, mobile, date, this, msg, is,...",spam
3,ham,"[this, is, wishing, you, a, great, day, moji, ...",ham
4,ham,"[thanks, again, for, your, reply, today, when,...",ham


In [ ]:
correct = (test_data['predicted'] == test_data['Label']).sum() / test_data.shape[0] * 100

In [ ]:
test_data.loc[test_data['predicted'] != test_data['Label']]

,Label,SMS,predicted
56,spam,"[money, i, have, won, wining, number, 946, wot...",ham
99,ham,"[gettin, rdy, to, ship, comp]",spam
142,ham,"[have, you, laid, your, airtel, line, to, rest]",spam
218,spam,"[hi, babe, its, chloe, how, r, u, i, was, smas...",ham
245,ham,[anytime],spam
404,ham,"[nokia, phone, is, lovly]",spam
491,spam,"[hi, this, is, amy, we, will, be, sending, you...",ham
588,ham,"[we, have, sent, jd, for, customer, service, c...",spam
646,ham,"[a, boy, loved, a, gal, he, propsd, bt, she, d...",needs human classification
912,spam,"[dating, i, have, had, two, of, these, only, s...",ham


In [ ]:
correct

99.10233393177738